In [1]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
    
    def forward(self, x, y):
        self.x = x
        self.y = y
        
        return x * y
    
    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x
        
        return dx, dy

In [2]:
apple = 100
apple_num = 2
tax = 1.1

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# layer
apple_price = mul_apple_layer.forward(apple, apple_num)
apple_price = mul_tax_layer.forward(apple_price, tax)

print(apple_price)

dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

220.00000000000003
2.2 110.00000000000001 200


In [3]:
class AddLayer:
    def __init__(self):
        pass
    
    def forward(self, x, y):
        self.x = x
        self.y = y
        
        return x + y
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        
        return dx, dy

In [4]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

#layer
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_layer = AddLayer()
mul_tax_layer = MulLayer()

# forwad
mul_apple_price = mul_apple_layer.forward(apple, apple_num)
mul_orange_price = mul_orange_layer.forward(orange, orange_num)
add_apple_orange_price = add_layer.forward(mul_apple_price, mul_orange_price)
mul_tax_price = mul_tax_layer.forward(add_apple_orange_price, tax)

print(mul_tax_price)

# backward
dprice = 1
dprice_all, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorg_price = add_layer.backward(dprice_all)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)
dorg, dorg_num = mul_orange_layer.backward(dorg_price)

print(dapple, dapple_num, dorg, dorg_num, dtax)

715.0000000000001
2.2 110.00000000000001 3.3000000000000003 165.0 650


In [5]:
class Relu:
    def __init__(self):
        self.mask = None
    
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        
        return dx

In [6]:
import numpy as np

x = np.array([[1.0, -0.5], [-2.0, 3.0]])
print(x)
relu = Relu()
relu.forward(x)

[[ 1.  -0.5]
 [-2.   3. ]]


array([[1., 0.],
       [0., 3.]])

In [7]:
class Sigmod:
    def __init__(self):
        self.out = None
    
    def forward(self, x):
        self.out = 1 / (1 + np.exp(-x))
        return self.out
        
    def backward(self, d):
        dx = d * (1 - self.out) * self.out
        
        return dx

In [8]:
x = np.array([[1.0, -0.5], [-2.0, 3.0]])
s = Sigmod()
f = s.forward(x)
b = s.backward(x)
print("forward: ", f)
print("backward: ", b)

forward:  [[0.73105858 0.37754067]
 [0.11920292 0.95257413]]
backward:  [[ 0.19661193 -0.11750186]
 [-0.20998717  0.13552998]]


In [9]:
X = np.random.rand(2)
W = np.random.rand(2, 3)
B = np.random.rand(3)
print(X)
print(X.shape)

[0.35009108 0.15794375]
(2,)


In [10]:
class Affine:
    def __init__(self, w, b):
        self.x = None
        self.W = w
        self.b = b
        self.dW = None
        self.db = None
    
    def forward(self, x):
        out = np.dot(x, self.W) + self.b
        self.x = x
        
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis = 0)
        
        return dx

In [11]:
def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a - c)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    
    return y

In [12]:
# y代表学习结果，t代表真实标签
def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t * np.log(y + delta))

In [13]:
class SoftmaxWithLoss:
    def __init__(self):
        self.y = None
        self.t = None
        self.loss = None
    
    def forward(self, x, t):
        self.y = softmax(x)
        self.t = t
        # 这里为什么是y和t呢？y代表学习结果，t代笔真实的分类
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss
        
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        
        return dx

In [14]:
def numerical_gradient(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x) # f(x+h)
        
        x[idx] = tmp_val - h 
        fxh2 = f(x) # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)
        
        x[idx] = tmp_val # 还原值
        it.iternext()   
        
    return grad

In [15]:
from collections import OrderedDict
class TwoLayerNetwork:
    def __init__(self, input_layer_size, hidden_layer_size, output_layer_size, weight_init_std=0.01):
        self.params = {}
        self.params["W1"] = weight_init_std * np.random.randn(input_layer_size, hidden_layer_size)
        self.params["b1"] = np.zeros(hidden_layer_size)
        self.params["W2"] = weight_init_std * np.random.randn(hidden_layer_size, output_layer_size)
        self.params["b2"] = np.zeros(output_layer_size)
        
        self.layers = OrderedDict()
        self.layers["Affine1"] = Affine(self.params["W1"], self.params["b1"])
        self.layers["Relu"] = Relu()
        self.layers["Affine2"] = Affine(self.params["W2"], self.params["b2"])
        
        self.lastlayers = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
    
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastlayers.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        # 为什么是argmax呢？
        y = np.argmax(y, axis=1)
        # 同问
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        accuracy = np.sum(y == t) / float(x.shape[0])
        
        return accuracy
    
    def numerical_gradient(self, x, t):
        loss_w = lambda W: self.loss(x, t)
        grads = {}
        grads["W1"] = numerical_gradient(loss_w, self.params["W1"])
        grads["b1"] = numerical_gradient(loss_w, self.params["b1"])
        grads["W2"] = numerical_gradient(loss_w, self.params["W2"])
        grads["b2"] = numerical_gradient(loss_w, self.params["b2"])
        
        return grads
        
    def gradient(self, x, t):
        # build network
        self.loss(x, t)
        dout = 1
        dout = self.lastlayers.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
        
        grads = {}
        grads["W1"] = self.layers["Affine1"].dW
        grads["b1"] = self.layers["Affine1"].db
        grads["W2"] = self.layers["Affine2"].dW
        grads["b2"] = self.layers["Affine2"].db
        
        return grads

In [16]:
import sys, os
sys.path.append(os.pardir)
from dataset.mnist import load_mnist
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
network = TwoLayerNetwork(input_layer_size=784, hidden_layer_size=50, output_layer_size=10)
x_batch = x_train[:3]
t_batch = t_train[:3]
print("x_batch: ", x_batch)
print("t_batch: ", t_batch)
print("shape of x_train: ", np.shape(x_train))

grad_num = network.numerical_gradient(x_batch, t_batch)
grad_back = network.gradient(x_batch, t_batch)

for key in grad_num.keys():
    diff = np.average(np.abs(grad_back[key] - grad_num[key]))
    print(key, ": ", diff)

x_batch:  [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
t_batch:  [[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]
shape of x_train:  (60000, 784)
b1 :  0.004606843199423458
b2 :  0.30558738236535954
W1 :  0.0008279328920732116
W2 :  0.014930895237113133


In [112]:
class TwoLayerNet_Ori:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 初始化权重
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) 
        self.params['b2'] = np.zeros(output_size)

        # 生成层
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
        
    # x:输入数据, t:监督数据
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # x:输入数据, t:监督数据
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
        
    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 设定
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads

In [119]:
x_batch = x_train[:30]
t_batch = t_train[:30]

network_ori = TwoLayerNet_Ori(784, 50, 10)
ori_accuracy = network_ori.accuracy(x_batch, t_batch)
print("ori_accuracy: ", ori_accuracy)
accuracy = network.accuracy(x_batch, t_batch)
print("accuracy: ", accuracy)

ori_accuracy:  0.13333333333333333
accuracy:  0.06666666666666667


In [121]:
import sys, os
sys.path.append(os.pardir)
from dataset.mnist import load_mnist
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
network_ori = TwoLayerNet_Ori(784, 50, 10)
x_batch = x_train[:3]
t_batch = t_train[:3]
print("x_batch: ", x_batch)
print("t_batch: ", t_batch)
grad_num = network_ori.numerical_gradient(x_batch, t_batch)
grad_back = network_ori.gradient(x_batch, t_batch)

for key in grad_num.keys():
    diff = np.average(np.abs(grad_back[key] - grad_num[key]))
    print(key, ": ", diff)

x_batch:  [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
t_batch:  [[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]
W1 :  0.0006911384331113362
b1 :  0.0038734796867109732
W2 :  0.011429342770965347
b2 :  0.3066646469540119
